In [ ]:
# This is Luke's scratchpad notebook.

In [1]:
# Importing the libraries:
import pandas as pd
import numpy as np
import math
from scipy import stats

# visualizing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# plt.rc('figure', figsize=(13, 10))
# plt.rc('font', size=14)

# preparing
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# modeling and evaluating
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix

# turn off warnings
import warnings
warnings.filterwarnings("ignore")

# acquiring
from pydataset import data

In [2]:
# SVI doesn't have zip codes, it has fips. So I need to transform FIPS into zip codes, and *then* I can join on the Covid cases onto the data frame

#### Key datasource:
https://www.huduser.gov/portal/datasets/usps_crosswalk.html
Possible note for presentation comments:
- we downloaded the 4th quarter 2018 data Tact-to-zip file, which is the file from HUD that'll translate the data

### Data Prep

In [3]:
import acquire
import prepare

In [4]:
import wrangle

In [ ]:
df = acquire.run()

In [ ]:
df.head()

In [ ]:
df = prepare.prepare_data(df)

In [ ]:
df.head()

# Moving into Explore

- Where are the clusters of cases visually?
- Where are the most dense clusters?
- How to define dense vs not dense?
- What are some other visually interesting things?
    - Scatterplot of all cases?
    - mapplot of svi by census tract?

In [5]:
df, train_exp, X_train_scaled, y_train, X_test_scaled, y_test = wrangle.wrangle_data()

Acquire: compiling raw data files...
Acquire: Completed!
Prepare: preparing data files...
Prepare: Completed!
(289, 25) (73, 25)


In [9]:
train_exp.head()

,tract,raw_svi,f_pov_soci,f_unemp_soci,f_pci_soci,f_nohsdp_soci,f_soci_total,f_age65_comp,f_age17_comp,f_disabl_comp,...,f_mobile_trans,f_crowd_trans,f_noveh_trans,f_groupq_trans,f_trans_total,all_flags_total,zip,tract_cases_per_100k,bin_svi,rank_svi
289,48029181821,0.4879,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,78240,2771.002224,low_mod,3
200,48029171902,0.6394,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,78245,2924.011251,mod_high,2
69,48029130200,0.6523,0,0,0,0,0,0,0,0,...,0,0,1,0,1,2,78203,4177.649457,mod_high,2
29,48029121120,0.0872,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,78247,2222.089837,low,4
12,48029120502,0.9531,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,78218,3235.404493,high,1


In [10]:
# do a bin of how many flags == 1 per tract

In [ ]:
# Get ideas from zillow project, linear regression explore lesson, 